In [2]:
import pandas as pd
import ast
import re

In [4]:
df_original = pd.read_csv('olympic_results.csv')

df_medalhas = df_original[df_original['medal_type'].notna() & 
                          df_original['medal_type'].isin(['GOLD', 'SILVER', 'BRONZE'])].copy()

def extrair_ano_inline(slug):
    if isinstance(slug, str):
        match = re.search(r'(\d{4})$', slug)
        if match:
            return int(match.group(1))
    return None

df_medalhas.loc[:, 'year'] = df_medalhas['slug_game'].apply(extrair_ano_inline)
df_medalhas.dropna(subset=['year'], inplace=True)
df_medalhas.loc[:, 'year'] = df_medalhas['year'].astype(int)

atletas_medalhas_lista = []

for _, row in df_medalhas.iterrows():
    year = row['year']
    country_name = row['country_name']
    medal_type = row['medal_type']
    atletas_str = row['athletes']
    
    if pd.notna(atletas_str) and isinstance(atletas_str, str) and atletas_str.strip().startswith("["):
        lista_atletas_tuplas = ast.literal_eval(atletas_str)
        for atleta_tupla in lista_atletas_tuplas:
            if isinstance(atleta_tupla, tuple) and len(atleta_tupla) > 0:
                athlete_name = atleta_tupla[0]
                atletas_medalhas_lista.append({
                    'year': year,
                    'country_name': country_name,
                    'athlete_name': athlete_name,
                    'medal_type': medal_type
                })
    elif pd.notna(row['athlete_full_name']) and row['participant_type'] == 'Athlete':
        atletas_medalhas_lista.append({
            'year': year,
            'country_name': country_name,
            'athlete_name': row['athlete_full_name'],
            'medal_type': medal_type
        })


df_saida = pd.DataFrame(atletas_medalhas_lista)
df_saida.to_csv('athlete_medals_by_edition.csv', index=False)